In [1]:
!pip install huggingface_hub
!pip install transformers
!pip install datasets
!pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
from datasets import load_dataset

# Load the dataset with the `large_random_1k` subset
dataset = load_dataset('poloclub/diffusiondb', '2m_first_1k')

In [3]:
my_1k_data = dataset['train']
my_1k_data

Dataset({
    features: ['image', 'prompt', 'seed', 'step', 'cfg', 'sampler', 'width', 'height', 'user_name', 'timestamp', 'image_nsfw', 'prompt_nsfw'],
    num_rows: 1000
})

In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import AutoTokenizer
import transformers
import torch
import random
import json
import re

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

#testing for one index
index = 0

samp = """I will give you a prompt. Generate 6 to 7 pairs of questions and answers from this prompt. You can include question about what attributes mentioned in the prompt such as shape, size, color etc. Think of these questions in such a way that their answers complete the description of the prompt and can be used to make an image through the features described in it. As an example, look at this prompt and the question answer pair generated for it:
Prompt: a little girl embroidery, fine art, muted background, calm colore. oil paiting by jan vermeer, masterpiese Question-Answer pair to be generated: Q. What is the object? A. The object is an embroidery of a little girl Q. What is the background color? A. The background is muted  Q.What type of painting is it? A. It is an oil painting. Q.Who is the painter? A. Jan Vermeer is the painter Generate similar question answer pairs for the following prompt:"""

sequences = pipeline(
    # samp + my_1k_data['prompt'][index] + "<Q&A>",
    samp + my_1k_data['prompt'][index] + "Question-Answer pair to be generated:",
    # samp + my_1k_data['prompt'][index] ,
    # my_1k_data['prompt'][index],
    do_sample=False,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    # max_length=200,
    max_new_tokens=512,
    temperature = 1e-05,
    top_p = 1,
    repetition_penalty = 1.0,
    length_penalty = 1
)

text = sequences[0]['generated_text']

input_text = text.split('Question-Answer pair to be generated:')
input_text

qa_dict = {}

# Iterate through array starting from the second index
for i in range(2, len(input_text), 2):
    # Extract questions and answers
    questions = re.findall(r'Q\..*?\?', input_text[i])
    answers = re.findall(r'A\..*?(?=(?:Q\.|$))', input_text[i])

    # Populate the dictionary with questions and answers
    for question, answer in zip(questions, answers):
        qa_dict[question.strip()] = answer.strip()

# Print the resulting dictionary
print(qa_dict)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/amudra/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{'Q. What is the subject of the portrait?': 'A. The subject of the portrait is Dwayne Johnson', 'Q. What is the style of the painting?': 'A. The painting is in the style of Rembrandt', 'Q. What is the medium used in the painting?': 'A. The medium used in the painting is oil on canvas', 'Q. What is the technique used in the painting?': 'A. The technique used in the painting is wet-on-wet', 'Q. What is the level of realism in the painting?': 'A. The level of realism in the painting is highly realistic', 'Q. What is the lighting effect in the painting?': 'A. The lighting effect in the painting is cinematic lightning', 'Q. What are the textures like in the painting?': 'A. The textures in the painting are intricate', 'Q. What is the level of detail in the painting?': 'A. The level of detail in the painting is highly intricate and illusionistic.'}


In [6]:
samp = """I will give you a prompt. Generate 6 to 7 pairs of questions and answers from this prompt. You can include question about what attributes mentioned in the prompt such as shape, size, color etc. Think of these questions in such a way that their answers complete the description of the prompt and can be used to make an image through the features described in it. As an example, look at this prompt and the question answer pair generated for it:
Prompt: a little girl embroidery, fine art, muted background, calm colore. oil paiting by jan vermeer, masterpiese Question-Answer pair to be generated: Q. What is the object? A. The object is an embroidery of a little girl Q. What is the background color? A. The background is muted  Q.What type of painting is it? A. It is an oil painting. Q.Who is the painter? A. Jan Vermeer is the painter Generate similar question answer pairs for the following prompt:"""

# Initialize the final output dictionary
final_output = {}

for index in range(min(3, len(my_1k_data['image']))):

    sequences = pipeline(
        samp + my_1k_data['prompt'][index] + "Question-Answer pair to be generated:",
        do_sample=False,
        top_k=50,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        temperature=1e-05,
        top_p=1,
        repetition_penalty=1.0,
        length_penalty=1
    )

    text = sequences[0]['generated_text']

    input_text = text.split('Question-Answer pair to be generated:')
    
    qa_dict = {}
    for i in range(2, len(input_text), 2):
        questions = re.findall(r'Q\..*?\?', input_text[i])
        answers = re.findall(r'A\..*?(?=(?:Q\.|$))', input_text[i])

        for question, answer in zip(questions, answers):
            qa_dict[question.strip()] = answer.strip()

    final_output[index] = {
        'image': my_1k_data['image'][index],
        'prompt': my_1k_data['prompt'][index],
        'qa_pairs': qa_dict
    }

print(final_output)

{0: {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x768 at 0x155363110950>, 'prompt': 'a renaissance portrait of dwayne johnson, art in the style of rembrandt!! intricate. ultra detailed, oil on canvas, wet - on - wet technique, pay attention to facial details, highly realistic, cinematic lightning, intricate textures, illusionistic detail, ', 'qa_pairs': {'Q. What is the subject of the portrait?': 'A. The subject of the portrait is Dwayne Johnson', 'Q. What is the style of the painting?': 'A. The painting is in the style of Rembrandt', 'Q. What is the medium used in the painting?': 'A. The medium used in the painting is oil on canvas', 'Q. What is the technique used in the painting?': 'A. The technique used in the painting is wet-on-wet', 'Q. What is the level of realism in the painting?': 'A. The level of realism in the painting is highly realistic', 'Q. What is the lighting effect in the painting?': 'A. The lighting effect in the painting is cinematic lightning', 

In [ ]:
######################################################################

In [ ]:
#####################################################################

In [ ]:
#Final Single Cell Code for QnA Gen of All Images in Dataset
#Can be implemented after Cell 4
#Input: my_1k_data (dataset)
#Output: final_output (dictionary)

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
import random
import json
import re

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

samp = """I will give you a prompt. Generate 6 to 7 pairs of questions and answers from this prompt. You can include question about what attributes mentioned in the prompt such as shape, size, color etc. Think of these questions in such a way that their answers complete the description of the prompt and can be used to make an image through the features described in it. As an example, look at this prompt and the question answer pair generated for it:
Prompt: a little girl embroidery, fine art, muted background, calm colore. oil paiting by jan vermeer, masterpiese Question-Answer pair to be generated: Q. What is the object? A. The object is an embroidery of a little girl Q. What is the background color? A. The background is muted  Q.What type of painting is it? A. It is an oil painting. Q.Who is the painter? A. Jan Vermeer is the painter Generate similar question answer pairs for the following prompt:"""

# Initialize the final output dictionary
final_output = {}

for index in range(len(my_1k_data['image'])):

    sequences = pipeline(
        samp + my_1k_data['prompt'][index] + "Question-Answer pair to be generated:",
        do_sample=False,
        top_k=50,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        temperature=1e-05,
        top_p=1,
        repetition_penalty=1.0,
        length_penalty=1
    )

    text = sequences[0]['generated_text']

    input_text = text.split('Question-Answer pair to be generated:')
    
    qa_dict = {}
    for i in range(2, len(input_text), 2):
        questions = re.findall(r'Q\..*?\?', input_text[i])
        answers = re.findall(r'A\..*?(?=(?:Q\.|$))', input_text[i])

        for question, answer in zip(questions, answers):
            qa_dict[question.strip()] = answer.strip()

    final_output[index] = {
        'image': my_1k_data['image'][index],
        'prompt': my_1k_data['prompt'][index],
        'qa_pairs': qa_dict
    }

print(final_output)


In [ ]:
################################################################

In [ ]:
################################################################

In [ ]:
#Blip for 10 images test from (final_output) dictionary

In [7]:
final_output

{0: {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x768>,
  'prompt': 'a renaissance portrait of dwayne johnson, art in the style of rembrandt!! intricate. ultra detailed, oil on canvas, wet - on - wet technique, pay attention to facial details, highly realistic, cinematic lightning, intricate textures, illusionistic detail, ',
  'qa_pairs': {'Q. What is the subject of the portrait?': 'A. The subject of the portrait is Dwayne Johnson',
   'Q. What is the style of the painting?': 'A. The painting is in the style of Rembrandt',
   'Q. What is the medium used in the painting?': 'A. The medium used in the painting is oil on canvas',
   'Q. What is the technique used in the painting?': 'A. The technique used in the painting is wet-on-wet',
   'Q. What is the level of realism in the painting?': 'A. The level of realism in the painting is highly realistic',
   'Q. What is the lighting effect in the painting?': 'A. The lighting effect in the painting is cinematic lightning',

In [8]:
for key, value in final_output.items():
    questions_list = [re.sub(r'^Q\. ', '', question) for question in value['qa_pairs'].keys() if question.startswith('Q.') and question.endswith('?')]
    final_output[key]['questions'] = questions_list

# Print the updated final_output
print(final_output)

{0: {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x768 at 0x155363110950>, 'prompt': 'a renaissance portrait of dwayne johnson, art in the style of rembrandt!! intricate. ultra detailed, oil on canvas, wet - on - wet technique, pay attention to facial details, highly realistic, cinematic lightning, intricate textures, illusionistic detail, ', 'qa_pairs': {'Q. What is the subject of the portrait?': 'A. The subject of the portrait is Dwayne Johnson', 'Q. What is the style of the painting?': 'A. The painting is in the style of Rembrandt', 'Q. What is the medium used in the painting?': 'A. The medium used in the painting is oil on canvas', 'Q. What is the technique used in the painting?': 'A. The technique used in the painting is wet-on-wet', 'Q. What is the level of realism in the painting?': 'A. The level of realism in the painting is highly realistic', 'Q. What is the lighting effect in the painting?': 'A. The lighting effect in the painting is cinematic lightning', 

In [12]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
# # url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
# # image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
# image = my_1k_data['image'][0]
# # prompttemplate = "Here are some  questions about the Image try to answer them :"
# # prompttemplate = "What is the subject of the portrait? What is the technique used in the painting?"
# # string = "1. What is the dragon's physical appearance?2. What is the dragon's wing type?3. What is the dragon's body scale type?4. What is the dragon's wing color?5. What is the dragon's body color?6. What is the dragon's overall appearance?7. What is the dragon's size?8. What is the dragon's habitat?9. What is the dragon's behavior?10. What is the dragon's personality?"
# string = "What is the subject of the portrait? What is the style of the painting? What is the medium used in the painting? What is the technique used in the painting?What is the level of realism in the painting?What is the lighting effect in the painting? What are the textures like in the painting? What is the level of detail in the painting?"
# # What is the level of realism in the painting?, What is the lighting effect in the painting?, What are the textures like in the painting?, What is the level of detail in the painting?"
# # inst = "answer each question separately"
# # Q. What is the technique used in the painting?, Q. What is the level of realism in the painting?, Q. What is the lighting effect in the painting?, Q. What are the textures like in the painting?, Q. What is the level of detail in the painting?"

# # inputs = processor(images=image, text=prompttemplate + modified_string, return_tensors="pt").to(device)
# inputs = processor(images=image, text= string , return_tensors="pt").to(device)

# outputs = model.generate(
#     **inputs,
#     do_sample=False,
#     num_beams=5,
#     max_length=256,
#     min_length=1,
#     top_p=0.9,
#     repetition_penalty=1.5,
#     length_penalty=1.0,
#     temperature=1,
# )
# generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
# print(generated_text)

/home/amudra/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


What is the subject of the portrait? The subject of the portrait is a bald man. 

The style of the painting is realistic. 

The medium used in the painting is oil on canvas. 

The technique used in the painting is oil painting. 

The level of realism in the painting is high. 

The lighting effect in the painting is dramatic, with a strong contrast between light and dark areas. 

The textures in the painting are smooth and well-defined. 

The level of detail in the painting is impressive, showcasing the artist's skill and attention to detail.


In [13]:
# generated_text

"What is the subject of the portrait? The subject of the portrait is a bald man. \n\nThe style of the painting is realistic. \n\nThe medium used in the painting is oil on canvas. \n\nThe technique used in the painting is oil painting. \n\nThe level of realism in the painting is high. \n\nThe lighting effect in the painting is dramatic, with a strong contrast between light and dark areas. \n\nThe textures in the painting are smooth and well-defined. \n\nThe level of detail in the painting is impressive, showcasing the artist's skill and attention to detail."

In [14]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
# # url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
# # image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
# image = my_1k_data['image'][1]
# # prompttemplate = "Here are some  questions about the Image try to answer them :"
# # prompttemplate = "What is the subject of the portrait? What is the technique used in the painting?"
# # string = "1. What is the dragon's physical appearance?2. What is the dragon's wing type?3. What is the dragon's body scale type?4. What is the dragon's wing color?5. What is the dragon's body color?6. What is the dragon's overall appearance?7. What is the dragon's size?8. What is the dragon's habitat?9. What is the dragon's behavior?10. What is the dragon's personality?"
# # string = "What is the subject of the portrait? What is the style of the painting? What is the medium used in the painting? What is the technique used in the painting?What is the level of realism in the painting?What is the lighting effect in the painting? What are the textures like in the painting? What is the level of detail in the painting?"
# # What is the level of realism in the painting?, What is the lighting effect in the painting?, What are the textures like in the painting?, What is the level of detail in the painting?"
# # inst = "answer each question separately"
# # Q. What is the technique used in the painting?, Q. What is the level of realism in the painting?, Q. What is the lighting effect in the painting?, Q. What are the textures like in the painting?, Q. What is the level of detail in the painting?"

# string = " Who are some of the artists that have inspired the artist?"

# # inputs = processor(images=image, text=prompttemplate + modified_string, return_tensors="pt").to(device)
# inputs = processor(images=image, text= string , return_tensors="pt").to(device)

# outputs = model.generate(
#     **inputs,
#     do_sample=False,
#     num_beams=5,
#     max_length=256,
#     min_length=1,
#     top_p=0.9,
#     repetition_penalty=1.5,
#     length_penalty=1.0,
#     temperature=1,
# )
# generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
# print(generated_text)

Frida Kahlo, Salvador Dali, Gustav Klimt


In [15]:
# generated_text

'Frida Kahlo, Salvador Dali, Gustav Klimt'

In [9]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
from PIL import Image
import requests

torch.manual_seed(0)
np.random.seed(0)

# device = "cuda" if torch.cuda.is_available() else "cpu"
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

for key, value in final_output.items():
    image = value['image']
    # display(image)
    questions = value.get('questions', [])
    # print(questions)

    # Loop over each question for the current image
    for question in questions:
        string = question
        print(string)
        # prompt = "answer this question about the image: "
        inputs = processor(images=image, text=string, return_tensors="pt")

        outputs = model.generate(
            **inputs,
            do_sample=False,
            num_beams=5,
            max_length=256,
            min_length=1,
            top_p=0.9,
            repetition_penalty=1.5,
            length_penalty=1.0,
            temperature=1,
        )

        generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        print(generated_text)

        
        if 'answers_blip' not in value:
            value['answers_blip'] = {}
        value['answers_blip'][question] = generated_text

# Print the updated final_output
print(final_output)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

What is the subject of the portrait?


/home/amudra/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The subject of the portrait is a bald man.
What is the style of the painting?
The style of the painting is oil on canvas.
What is the medium used in the painting?
The medium used in the painting is oil on canvas.
What is the technique used in the painting?
The technique used in the painting is oil painting.
What is the level of realism in the painting?
The level of realism in the painting is high.
What is the lighting effect in the painting?
The lighting effect in the painting is a combination of natural and artificial lighting. The man's face appears to be illuminated by both sunlight and artificial light sources, such as lamps or spotlights.
What are the textures like in the painting?
The textures in the painting are very realistic and lifelike, capturing the details of the man's skin, hair, and facial features.
What is the level of detail in the painting?
The level of detail in the painting is quite high, as it depicts a close-up view of a bald man's face. The painting captures the 

In [10]:
final_output

{0: {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x768>,
  'prompt': 'a renaissance portrait of dwayne johnson, art in the style of rembrandt!! intricate. ultra detailed, oil on canvas, wet - on - wet technique, pay attention to facial details, highly realistic, cinematic lightning, intricate textures, illusionistic detail, ',
  'qa_pairs': {'Q. What is the subject of the portrait?': 'A. The subject of the portrait is Dwayne Johnson',
   'Q. What is the style of the painting?': 'A. The painting is in the style of Rembrandt',
   'Q. What is the medium used in the painting?': 'A. The medium used in the painting is oil on canvas',
   'Q. What is the technique used in the painting?': 'A. The technique used in the painting is wet-on-wet',
   'Q. What is the level of realism in the painting?': 'A. The level of realism in the painting is highly realistic',
   'Q. What is the lighting effect in the painting?': 'A. The lighting effect in the painting is cinematic lightning',

In [ ]:
########################################################

In [ ]:
########################################################

In [ ]:
########################################################

In [ ]:
#Complete run for 10 images 

In [ ]:
samp = """I will give you a prompt. Generate 6 to 7 pairs of questions and answers from this prompt. You can include question about what attributes mentioned in the prompt such as shape, size, color etc. Think of these questions in such a way that their answers complete the description of the prompt and can be used to make an image through the features described in it. As an example, look at this prompt and the question answer pair generated for it:
Prompt: a little girl embroidery, fine art, muted background, calm colore. oil paiting by jan vermeer, masterpiese Question-Answer pair to be generated: Q. What is the object? A. The object is an embroidery of a little girl Q. What is the background color? A. The background is muted  Q.What type of painting is it? A. It is an oil painting. Q.Who is the painter? A. Jan Vermeer is the painter Generate similar question answer pairs for the following prompt:"""

# Initialize the final output dictionary
final_output = {}

for index in range(min(10, len(my_1k_data['image']))):

    sequences = pipeline(
        samp + my_1k_data['prompt'][index] + "Question-Answer pair to be generated:",
        do_sample=False,
        top_k=50,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        temperature=1e-05,
        top_p=1,
        repetition_penalty=1.0,
        length_penalty=1
    )

    text = sequences[0]['generated_text']

    input_text = text.split('Question-Answer pair to be generated:')
    
    qa_dict = {}
    for i in range(2, len(input_text), 2):
        questions = re.findall(r'Q\..*?\?', input_text[i])
        answers = re.findall(r'A\..*?(?=(?:Q\.|$))', input_text[i])

        for question, answer in zip(questions, answers):
            qa_dict[question.strip()] = answer.strip()

    final_output[index] = {
        'image': my_1k_data['image'][index],
        'prompt': my_1k_data['prompt'][index],
        'qa_pairs': qa_dict
    }

print(final_output)

#Extracting questions
for key, value in final_output.items():
    questions_list = [re.sub(r'^Q\. ', '', question) for question in value['qa_pairs'].keys() if question.startswith('Q.') and question.endswith('?')]
    final_output[key]['questions'] = questions_list

print(final_output)

#Running InstructBlip
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
from PIL import Image
import requests

torch.manual_seed(0)
np.random.seed(0)

# device = "cuda" if torch.cuda.is_available() else "cpu"
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

for key, value in final_output.items():
    image = value['image']
    # display(image)
    questions = value.get('questions', [])
    # print(questions)

    # Loop over each question for the current image
    for question in questions:
        string = question
        print(string)
        # prompt = "answer this question about the image: "
        inputs = processor(images=image, text=string, return_tensors="pt")

        outputs = model.generate(
            **inputs,
            do_sample=False,
            num_beams=5,
            max_length=256,
            min_length=1,
            top_p=0.9,
            repetition_penalty=1.5,
            length_penalty=1.0,
            temperature=1,
        )

        generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        print(generated_text)

        
        if 'answers_blip' not in value:
            value['answers_blip'] = {}
        value['answers_blip'][question] = generated_text

# Print the updated final_output
print(final_output)